## Weekly Stock Pick
This should be a self contained notebook which can be run top to bottom each week. The output should be 5-10 companies which can be logged and their performance recorded as a paper test. The notebook should be run on a weekend, the idea being that if productionised, it would be run friday night before market close.
The order of operations should be:
1) Scrape historic data, save to a holding directory
2) Resample the data to be weekly such that the entire current week is considered as the most up to date data point
3) Attach relevant technical indicators and resave in another directory. This should be a modular process in case I want to add technical indicators in the future. Currently the RSI, MFI, ULT and my personal volume ranker are used.
4) For the most recent month, rank order all companies by the indicator scores, take 3 from each and combine them for unique codes


In [1]:
import os
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import shutil

### Step 1 -> Tickers
The ticker symbols for consideration should be all companies listed on the NYSE and nasdaq exchanges. The web link contains a download button for these tickers. The file should be renamed and placed in the working directory. They don't tend to change much so this should'nt need to be redone very often.

In [3]:
#https://www.nasdaq.com/market-activity/stocks/screener

# Load the CSV file into a DataFrame
df = pd.read_csv('tickers.csv')

# Extract the 'Symbol' column into a list
ticker_symbols = df['Symbol'].tolist()


### Step 2 -> Scraping
The list of tickers is north of 7,000 so we have chunked it up to give some visibile of the progress for the download but also for fault tolerance as some of the tickers are obscure and can throw errors. What we should get out of this is a directory full of files containing the typical daily market data including opens, closes, highs, lows and volumes. Important note for this code is that anything with an odd symbol is not being handled. There will be many symbols with ^ or / inside the string which don't work, some may not have any data, some may be delisted.

In [ ]:
"""
    Functions
"""
# Function to fetch and save data for a list of tickers
def fetch_and_save_data(tickers):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)
    
    try:
        #print(f"Fetching data for tickers: {tickers}")
        data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
        
        for ticker in tickers:
            ticker_data = data[ticker]
            if not ticker_data.empty:
                file_path = os.path.join(data_dir, f"{ticker}.csv")
                ticker_data.to_csv(file_path)
                print(f"Data for {ticker} saved successfully.")
            else:
                print(f"No data found for {ticker}.")
    except Exception as e:
        print(f"Error fetching data for tickers: {e}")

# Function to split the list into chunks of specified size
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

"""
    Driver code
"""

# Directory to save the data
data_dir = "ticker_data"
os.makedirs(data_dir, exist_ok=True)

# Process tickers in chunks of 500
chunk_size = 500
for ticker_chunk in chunk_list(ticker_symbols, chunk_size):
    fetch_and_save_data(ticker_chunk)

print("Data fetching completed.")

### Step 3 -> Resampling
The data we have scraped is daily. There are a few issues with this. One issue is that there is no guarantee that any ticker has the same days in it as any other. Some may have data which ends sooner or starts later. To get around most of the issues, we can use a merge function on the date index to combine them and have nulls for the missings, which is fine for our purposes. Additionally, we dont want to process daily data as its less useful to weekly resampling gives us appropriate data for the sort of technical indicators we are playing with. Lastly, we want the resample to cap off at the most recent Friday so final date in the output should be the last Friday.

In [21]:
import os
import pandas as pd

def process_ticker_data(directory: str):
    # Step 1: Load all CSV files into DataFrames
    dataframes = {}
    ticker_symbols = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            ticker = file.split('.csv')[0]
            ticker_symbols.append(ticker)
            filepath = os.path.join(directory, file)
            df = pd.read_csv(filepath, parse_dates=['Date'])
            dataframes[ticker] = df
    
    # Step 2: Create New Column Names
    for ticker, df in dataframes.items():
        df.rename(columns={
            'Open': f'{ticker}_Open',
            'High': f'{ticker}_High',
            'Low': f'{ticker}_Low',
            'Close': f'{ticker}_Close',
            'Adj Close': f'{ticker}_Adj_Close',
            'Volume': f'{ticker}_Volume'
        }, inplace=True)
    
    # Step 3: Merge DataFrames on the 'Date' Column
    merged_df = None
    for ticker, df in dataframes.items():
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
    # Step 4: Resample the DataFrame Weekly (ending on Fridays)
    merged_df.set_index('Date', inplace=True)
    weekly_df = merged_df.resample('W-FRI').last().reset_index()
    
    # Step 5: Split and Save DataFrames
    for ticker in ticker_symbols:
        ticker_columns = [col for col in weekly_df.columns if (col.split('_')[0] == ticker)]
        ticker_columns.insert(0, 'Date')  # Ensure 'Date' column is included
        ticker_df = weekly_df[ticker_columns]
        output_filepath = os.path.join('weekly_data', f'{ticker}.csv')
        ticker_df.to_csv(output_filepath, index=False)

# Example usage:
process_ticker_data('ticker_data')


KeyboardInterrupt: 

### Step 4 -> Technical Indicators
We need standardised measures of stocks which can be rank ordered in a way where the ranking of the indicator relative to the wider market gives us value. Good example is RSI where the most overbought companies in the Nasdaq and NYSE have significant positive sentiment behind them when the data was taken. Backtests verify that this sentiment tends to continue more often than it reverses so using it as a leading indicator of stock price growth works over the long term. MFI is a normalised version of my original indicator which just multiplies volume into adj close and rank orders.

In [ ]:
"""
    Functions
"""

def calculate_rsi(df, ticker, period=14):
    column = f'{ticker}_Close'
    delta = df[column].diff()
    gain = delta.clip(lower=0).rolling(window=period, min_periods=1).mean()
    loss = -delta.clip(upper=0).rolling(window=period, min_periods=1).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_mfi(df, ticker, period=14):
    typical_price = (df[f'{ticker}_High'] + df[f'{ticker}_Low'] + df[f'{ticker}_Close']) / 3
    money_fLow = typical_price * df[f'{ticker}_Volume']
    positive_fLow = money_fLow.where(typical_price > typical_price.shift(1), 0).rolling(window=period, min_periods=1).sum()
    negative_fLow = money_fLow.where(typical_price < typical_price.shift(1), 0).rolling(window=period, min_periods=1).sum()
    mfi = 100 - (100 / (1 + positive_fLow / negative_fLow))
    return mfi

def calculate_ultimate_oscillator(df, ticker, short_period=7, mid_period=14, long_period=28):
    # Ensure the DataFrame is sorted by date
    df = df.sort_index()

    # Calculate Buying Pressure (BP) and True Range (TR)
    df['Prior Close'] = df[f'{ticker}_Close'].shift(1)
    df['BP'] = df[f'{ticker}_Close'] - df[[f'{ticker}_Low', 'Prior Close']].min(axis=1)
    df['TR'] = df[[f'{ticker}_High', 'Prior Close']].max(axis=1) - df[[f'{ticker}_Low', 'Prior Close']].min(axis=1)
    
    # Calculate average BP and TR for each period
    df['Avg7_BP'] = df['BP'].rolling(window=short_period).sum()
    df['Avg7_TR'] = df['TR'].rolling(window=short_period).sum()
    df['Avg14_BP'] = df['BP'].rolling(window=mid_period).sum()
    df['Avg14_TR'] = df['TR'].rolling(window=mid_period).sum()
    df['Avg28_BP'] = df['BP'].rolling(window=long_period).sum()
    df['Avg28_TR'] = df['TR'].rolling(window=long_period).sum()
    
    # Calculate raw UO components
    df['R1'] = df['Avg7_BP'] / df['Avg7_TR']
    df['R2'] = df['Avg14_BP'] / df['Avg14_TR']
    df['R3'] = df['Avg28_BP'] / df['Avg28_TR']
    
    # Calculate Ultimate Oscillator
    df['Ultimate Oscillator'] = 100 * (4 * df['R1'] + 2 * df['R2'] + df['R3']) / (4 + 2 + 1)
    
    return df['Ultimate Oscillator']

def calculate_obv(df, ticker):
    # Calculate daily returns
    df['Daily Return'] = df[f'{ticker}_Close'].diff()
    
    # Calculate the direction of the volume flow
    df['Direction'] = 0
    df.loc[df['Daily Return'] > 0, 'Direction'] = 1
    df.loc[df['Daily Return'] < 0, 'Direction'] = -1
    
    # Calculate OBV
    df['OBV'] = (df[f'{ticker}_Volume'] * df['Direction']).cumsum()
    
    
    return df['OBV']
    
def calculate_close_open_avg_volume(df, ticker):
    # Calculate the average of 'Close' and 'Open' prices
    df['Close_Open_Avg'] = (df[f'{ticker}_Close'] + df[f'{ticker}_Open']) / 2
    
    # Multiply the average with 'Volume'
    df['Close_Open_Avg_Volume'] = df['Close_Open_Avg'] * df[f'{ticker}_Volume']
    
    return df['Close_Open_Avg_Volume']
    
def add_technical_indicators():
    weekly_data_dir = 'weekly_data'
    ta_data_dir = 'ta_data'
    
    if not os.path.exists(ta_data_dir):
        os.makedirs(ta_data_dir)
    
    # List all CSV files in the weekly_data directory
    ticker_files = [f for f in os.listdir(weekly_data_dir) if f.endswith('.csv')]
    
    for file in ticker_files:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(weekly_data_dir, file))
        
        if df.empty:
            continue
        
        # Ensure the date column is in datetime format
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Sort the dataframe by date
        df.sort_values(by='Date', inplace=True)

        ticker = file.split('.')[0]
        print(ticker)
        
        # Calculate technical indicators
        df['RSI'] = calculate_rsi(df, ticker)
        df['MFI'] = calculate_mfi(df, ticker)
        df['ULTOSC'] = calculate_ultimate_oscillator(df, ticker)
        df['OBV'] = calculate_obv(df, ticker)
        df['MON'] = calculate_close_open_avg_volume(df, ticker)
        
        
        # Remove any rows with NaN values introduced by the indicators
        df.dropna(inplace=True)

        # Only take the columns we need for writing
        final_df = df[['Date', 'RSI', 'MFI', 'ULTOSC', 'OBV', 'MON', f'{ticker}_Adj_Close']]
        
        final_df.rename(columns={col: 'Close' if col.endswith('Adj_Close') else col for col in final_df.columns}, inplace=True)
        
        # Save the updated DataFrame to the ta_data directory
        final_df.to_csv(os.path.join(ta_data_dir, file), index=False)


"""
    Driver code
"""

# Call the function to execute the task
add_technical_indicators()


### Step 5 -> Rank Ordering
This step loads in the previously saved data with the technical indicators attached. Now that we have this data across a standard date dimension and resampled at the weekly level, the highest number across each date can be observed as the hot stock of the day. When this is done, many small and tiny cap companies pop up as the small volume produces false signals when small amounts of money are used to buy or sell. We introduce an arbitrary minimum spend for the past week of 1 million dollars to remove these. What is left is the companies with the most positive sentiment against them which we then output.

In [ ]:
"""

def load_ta_data():
    ta_data_dir = 'ta_data'
    
    # List all CSV files in the ta_data directory
    ticker_files = [f for f in os.listdir(ta_data_dir) if f.endswith('.csv')]
    
    # List to store DataFrames
    dataframes = []
    
    for file in ticker_files:
        # Extract the ticker symbol from the file name (assuming it's the file name without extension)
        ticker_symbol = os.path.splitext(file)[0].split('_')[0]
        # Load the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(ta_data_dir, file))
        
        if df.empty:
            continue
        
        # Add a column for the ticker symbol
        df['ticker'] = ticker_symbol
        
        # Remove rows where 'MON' column is less than 1 million
        df = df[df['MON'] >= 1_000_000]   
        
        # Append the DataFrame to the list
        dataframes.append(df)
    
    # Concatenate all DataFrames
    combined_df = pd.concat(dataframes)
    
    # Ensure the date column is in datetime format
    combined_df['Date'] = pd.to_datetime(combined_df['Date'])
    
    # Set the date column as the index
    combined_df.set_index('Date', inplace=True)
    
    return combined_df



# Load the data
combined_df = load_ta_data()
#print(combined_df)

"""

In [15]:

# Directory containing the CSV files
directory = 'ta_data'

# Initialize an empty list to hold dataframes
dataframes = []

# Loop through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        ticker_symbol = filename.split('.')[0]
        df = pd.read_csv(filepath)
        df['Ticker'] = ticker_symbol
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df.loc[merged_df['MON'] < 10000000, :] = None
# Initialize an empty list to store the summary data
summary_data = []

# Group by 'Date' and find the top 5 ticker symbols for each specified column
for date, group in merged_df.groupby('Date'):
    top_rsi = group.nlargest(5, 'RSI')[['Ticker', 'RSI']]
    top_mfi = group.nsmallest(5, 'MFI')[['Ticker', 'MFI']]
    top_ultosc = group.nlargest(5, 'ULTOSC')[['Ticker', 'ULTOSC']]
    top_obv = group.nlargest(5, 'OBV')[['Ticker', 'OBV']]
    top_mon = group.nlargest(5, 'MON')[['Ticker', 'MON']]
    
    # Create a summary dictionary for the current date
    summary = {
        'Date': date,
        'Top_RSI': top_rsi['Ticker'].tolist(),
        'Top_MFI': top_mfi['Ticker'].tolist(),
        'Top_ULTOSC': top_ultosc['Ticker'].tolist(),
        'Top_OBV': top_obv['Ticker'].tolist(),
        'Top_MON': top_mon['Ticker'].tolist()
    }
    
    # Append the summary to the list
    summary_data.append(summary)

# Convert the summary data list to a dataframe
summary_df = pd.DataFrame(summary_data)
print(summary_df)


C:\Users\lcarp\AppData\Local\Temp\ipykernel_7684\2637258726.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(dataframes, ignore_index=True)


           Date                        Top_RSI  \
0    2014-12-26  [ROST, BBWI, WELL, EQR, ORLY]   
1    2015-01-02     [WELL, SKT, ROST, HA, LOW]   
2    2015-01-09    [WELL, SKT, SPG, ROST, EQR]   
3    2015-01-16    [WELL, SKT, SHW, SPG, BERY]   
4    2015-01-23       [LOW, SPG, KR, SHW, UAL]   
..          ...                            ...   
490  2024-05-17    [PPC, VITL, VIRT, AEM, HBM]   
491  2024-05-24   [VITL, LPG, FIP, PRMW, VIRT]   
492  2024-05-31      [VITL, LPG, KGC, AA, FSM]   
493  2024-06-07   [VITL, AGX, PRMW, AZN, NVAX]   
494  2024-06-14     [EDR, VITL, AGX, HEI, AZN]   

                            Top_MFI                      Top_ULTOSC  \
0      [KEX, EQNR, GOLD, HAL, FSLR]      [TPST, MAC, FTNT, LOW, HE]   
1         [RES, BBVA, AY, PDS, KEX]       [TPST, MAC, HE, KMX, PNW]   
2       [TAL, ABUS, BBVA, RES, PDS]       [TPST, HE, TNK, SUI, ELS]   
3       [RYAM, WDS, RES, ASPS, OBE]    [TPST, TCRT, WELL, RMD, ESS]   
4      [RYAM, SRV, ASPS, VSTM, SXC]  [LULU, 

### Step 6 -> Output


In [24]:
# Sort the dataframe by 'Ticker' and 'Date'
merged_df.sort_values(by=['Ticker', 'Date'], inplace=True)

# Calculate the percentage difference in 'Close' column for each ticker
merged_df['Close_Pct_Diff'] = merged_df.groupby('Ticker')['Close'].pct_change() * 100
merged_df['Close_Pct_Diff'] = merged_df['Close_Pct_Diff'].shift(1)

# Initialize an empty list to store the summary data with percentage differences
summary_pct_diff_data = []

# Loop through each date and the corresponding top tickers from the previous summary
for index, row in summary_df.iterrows():
    date = row['Date']
    
    # Extract the top tickers for each indicator on the current date
    top_rsi_tickers = row['Top_RSI']
    top_mfi_tickers = row['Top_MFI']
    top_ultosc_tickers = row['Top_ULTOSC']
    top_obv_tickers = row['Top_OBV']
    top_mon_tickers = row['Top_MON']
    
    # Find the percentage differences for the top tickers
    top_rsi_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_rsi_tickers))][['Ticker', 'Close_Pct_Diff']].dropna()
    top_mfi_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_mfi_tickers))][['Ticker', 'Close_Pct_Diff']].dropna()
    top_ultosc_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_ultosc_tickers))][['Ticker', 'Close_Pct_Diff']].dropna()
    top_obv_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_obv_tickers))][['Ticker', 'Close_Pct_Diff']].dropna()
    top_mon_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_mon_tickers))][['Ticker', 'Close_Pct_Diff']].dropna()
    
    # Create a summary dictionary for the current date with percentage differences
    summary_pct_diff = {
        'Date': date,
        'Top_RSI_Pct_Diff': top_rsi_pct_diff.set_index('Ticker')['Close_Pct_Diff'].to_dict(),
        'Top_MFI_Pct_Diff': top_mfi_pct_diff.set_index('Ticker')['Close_Pct_Diff'].to_dict(),
        'Top_ULTOSC_Pct_Diff': top_ultosc_pct_diff.set_index('Ticker')['Close_Pct_Diff'].to_dict(),
        'Top_OBV_Pct_Diff': top_obv_pct_diff.set_index('Ticker')['Close_Pct_Diff'].to_dict(),
        'Top_MON_Pct_Diff': top_mon_pct_diff.set_index('Ticker')['Close_Pct_Diff'].to_dict()
    }
    
    # Append the summary with percentage differences to the list
    summary_pct_diff_data.append(summary_pct_diff)

# Convert the summary data with percentage differences list to a dataframe
summary_pct_diff_df = pd.DataFrame(summary_pct_diff_data)

print(summary_pct_diff_df)

           Date                                   Top_RSI_Pct_Diff  \
0    2014-12-26  {'BBWI': -9.032164691383137, 'EQR': -1.8840596...   
1    2015-01-02                       {'SKT': -0.3261868490705755}   
2    2015-01-09  {'EQR': -0.24461878739424625, 'ROST': 0.342594...   
3    2015-01-16  {'BERY': -0.15754349091625652, 'SHW': 4.564814...   
4    2015-01-23  {'KR': 1.3669211790297453, 'LOW': -0.151445084...   
..          ...                                                ...   
490  2024-05-17  {'AEM': 5.166044302393513, 'HBM': 4.8984450778...   
491  2024-05-24  {'FIP': 5.851759203561668, 'LPG': 1.2972003637...   
492  2024-05-31  {'AA': 1.2645925857757145, 'FSM': 4.1095850145...   
493  2024-06-07  {'AGX': 0.15598499090736517, 'AZN': -0.6620884...   
494  2024-06-14  {'AGX': 8.098543739397247, 'AZN': 2.5634453807...   

                                      Top_MFI_Pct_Diff  \
0    {'EQNR': 1.7082778994878423, 'FSLR': -5.286130...   
1    {'AY': 17.684894189294397, 'BBVA': 3.2

In [25]:
# Initialize an empty list to store the summary data with average percentage changes
summary_avg_pct_diff_data = []

# Loop through each date and the corresponding top tickers from the previous summary
for index, row in summary_df.iterrows():
    date = row['Date']
    
    # Extract the top tickers for each indicator on the current date
    top_rsi_tickers = row['Top_RSI']
    top_mfi_tickers = row['Top_MFI']
    top_ultosc_tickers = row['Top_ULTOSC']
    top_obv_tickers = row['Top_OBV']
    top_mon_tickers = row['Top_MON']
    
    # Find the percentage differences for the top tickers and calculate the average
    top_rsi_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_rsi_tickers))]['Close_Pct_Diff'].dropna().mean()
    top_mfi_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_mfi_tickers))]['Close_Pct_Diff'].dropna().mean()
    top_ultosc_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_ultosc_tickers))]['Close_Pct_Diff'].dropna().mean()
    top_obv_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_obv_tickers))]['Close_Pct_Diff'].dropna().mean()
    top_mon_pct_diff = merged_df[(merged_df['Date'] == date) & (merged_df['Ticker'].isin(top_mon_tickers))]['Close_Pct_Diff'].dropna().mean()
    
    # Create a summary dictionary for the current date with average percentage differences
    summary_avg_pct_diff = {
        'Date': date,
        'Avg_RSI_Pct_Diff': top_rsi_pct_diff,
        'Avg_MFI_Pct_Diff': top_mfi_pct_diff,
        'Avg_ULTOSC_Pct_Diff': top_ultosc_pct_diff,
        'Avg_OBV_Pct_Diff': top_obv_pct_diff,
        'Avg_MON_Pct_Diff': top_mon_pct_diff
    }
    
    # Append the summary with average percentage differences to the list
    summary_avg_pct_diff_data.append(summary_avg_pct_diff)

# Convert the summary data with average percentage differences list to a dataframe
summary_avg_pct_diff_df = pd.DataFrame(summary_avg_pct_diff_data)
print(summary_avg_pct_diff_df)


           Date  Avg_RSI_Pct_Diff  Avg_MFI_Pct_Diff  Avg_ULTOSC_Pct_Diff  \
0    2014-12-26         -3.914299          5.811881            11.567331   
1    2015-01-02         -0.326187          8.416426                  NaN   
2    2015-01-09          0.172725         -9.013236             0.541593   
3    2015-01-16          3.517133         -6.280910             4.943277   
4    2015-01-23          0.755258         -7.864028             3.343370   
..          ...               ...               ...                  ...   
490  2024-05-17          8.332611         -9.492014            12.026822   
491  2024-05-24          3.261113         -2.398849             5.456885   
492  2024-05-31          4.770342         -4.173599            10.433224   
493  2024-06-07         -0.367699         -2.330124             1.735437   
494  2024-06-14          2.447350         -5.375610             1.744256   

     Avg_OBV_Pct_Diff  Avg_MON_Pct_Diff  
0          -29.327979        -13.246538  
1  

In [32]:
summary_avg_pct_diff_df['Avg_ULTOSC_Pct_Diff'].mean()


7.728679251335436

## Purge the Folders
Can't push this much data to source control so once we have our output file, we remove the contents of the three folders we've been holding data in. Uncomment to run

In [33]:

def purge_directories(directories):
    for directory in directories:
        if os.path.exists(directory):
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f'Failed to delete {file_path}. Reason: {e}')

# Directories to be purged
directories = ['ta_data', 'ticker_data', 'weekly_data']

# Purge the directories
purge_directories(directories)
